# 096: Batch Processing at Scale

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import List, Dict, Any, Callable
from dataclasses import dataclass, field
import time
import multiprocessing as mp
from functools import partial

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

print("✅ Batch processing environment ready!")
print(f"Available CPU cores: {mp.cpu_count()}")
print("\nProduction Tools:")
print("  - Apache Spark: Distributed batch processing")
print("  - Apache Hadoop: MapReduce framework")
print("  - Dask: Python parallel computing")
print("  - Databricks: Managed Spark platform")

### 📝 What's Happening in This Code?

**Purpose:** Set up environment for distributed batch processing simulation

**Key Points:**
- **Multiprocessing**: Python simulates distributed compute (actual parallelism on multi-core CPU)
- **Production Reality**: Spark/Hadoop run on 100-1000 node clusters (6400+ cores)
- **CPU Cores**: Laptop 8-16, cloud 32-96, cluster 1000s
- **Frameworks**: Spark 100× faster than Hadoop MapReduce

**Why This Matters:** Intel runs 200-node Spark clusters processing 500TB/week. AWS EMR charges $0.10/core-hour → optimize to finish faster!

## 2. Data Partitioning Strategies

In [ ]:
@dataclass
class DataPartition:
    """Represents a partition of data for distributed processing"""
    partition_id: int
    data: pd.DataFrame
    partition_key: str
    
    @property
    def size_mb(self) -> float:
        return self.data.memory_usage(deep=True).sum() / 1024 / 1024
    
    @property
    def row_count(self) -> int:
        return len(self.data)

# Generate test data (1M rows simulating test results)
np.random.seed(42)
n_rows = 1_000_000

df_test_data = pd.DataFrame({
    'device_id': [f"D{np.random.randint(1, 10001):05d}" for _ in range(n_rows)],
    'wafer_id': [f"W{np.random.randint(1, 51):03d}" for _ in range(n_rows)],
    'test_name': np.random.choice(['Vdd', 'Idd', 'Freq', 'Power'], n_rows),
    'test_value': np.random.normal(100, 10, n_rows),
    'test_time_ms': np.random.normal(50, 10, n_rows),
    'timestamp': [datetime(2024, 1, 1) + timedelta(seconds=i) for i in range(n_rows)]
})

print(f"📊 Generated {len(df_test_data):,} test results")
print(f"   Size: {df_test_data.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
print(f"   Unique wafers: {df_test_data['wafer_id'].nunique()}")
print(f"   Unique devices: {df_test_data['device_id'].nunique()}")

### 📝 What's Happening in This Code?

**Purpose:** Create test dataset simulating semiconductor test results

**Key Points:**
- **1M Rows**: Realistic sample size for local testing (production: billions)
- **Realistic Data**: device_id, wafer_id, test parameters (Vdd, Idd, Freq, Power)
- **Memory Size**: ~80MB uncompressed (production: 500TB Parquet compressed 10×)
- **DataPartition Class**: Track partition metadata (size, row count)

**Why This Matters:** Intel's 500TB = 6 billion rows. Partition into 4000 chunks (125GB each) for distributed processing.

## 3. Hash Partitioning Implementation

In [ ]:
class HashPartitioner:
    """Hash partitioning for even distribution"""
    
    @staticmethod
    def partition(df: pd.DataFrame, key: str, num_partitions: int) -> List[DataPartition]:
        """Distribute data evenly using hash function"""
        df['_partition_id'] = df[key].apply(lambda x: hash(str(x)) % num_partitions)
        
        partitions = []
        for i in range(num_partitions):
            partition_data = df[df['_partition_id'] == i].drop('_partition_id', axis=1)
            partitions.append(DataPartition(
                partition_id=i,
                data=partition_data,
                partition_key=key
            ))
        
        return partitions

# Test hash partitioning
num_partitions = 8
hash_partitions = HashPartitioner.partition(df_test_data, 'wafer_id', num_partitions)

print(f"🔀 Hash Partitioning (by wafer_id, {num_partitions} partitions):\n")
for p in hash_partitions:
    print(f"  Partition {p.partition_id}: {p.row_count:,} rows ({p.size_mb:.2f} MB)")

# Check balance
sizes = [p.row_count for p in hash_partitions]
print(f"\n  Balance: min={min(sizes):,}, max={max(sizes):,}, "
      f"stddev={np.std(sizes):.0f} rows")
print(f"  Skew: {(max(sizes) - min(sizes)) / np.mean(sizes) * 100:.1f}%")

### 📝 What's Happening in This Code?

**Purpose:** Implement hash partitioning for even data distribution

**Key Points:**
- **Hash Function**: `hash(key) % N` → distributes data evenly across N partitions
- **Good For**: Joins, groupBy (all records with same key in same partition)
- **Balance Check**: Stddev and skew metrics (want <5% skew)
- **Target Size**: 128-512MB per partition in Spark (avoid small/large extremes)

**Why This Matters:** Intel partitions 500TB by wafer_id (50K wafers = 10GB/partition). Skewed partitions cause stragglers (one task 10× slower → entire job delayed).

## 4. Range Partitioning Implementation

In [ ]:
class RangePartitioner:
    """Range partitioning for time-series and sorted data"""
    
    @staticmethod
    def partition(df: pd.DataFrame, key: str, num_partitions: int) -> List[DataPartition]:
        """Partition based on value ranges"""
        df_sorted = df.sort_values(key)
        partition_size = len(df_sorted) // num_partitions
        
        partitions = []
        for i in range(num_partitions):
            start_idx = i * partition_size
            end_idx = start_idx + partition_size if i < num_partitions - 1 else len(df_sorted)
            partition_data = df_sorted.iloc[start_idx:end_idx]
            
            partitions.append(DataPartition(
                partition_id=i,
                data=partition_data,
                partition_key=key
            ))
        
        return partitions

# Test range partitioning
range_partitions = RangePartitioner.partition(df_test_data, 'timestamp', num_partitions)

print(f"📅 Range Partitioning (by timestamp, {num_partitions} partitions):\n")
for p in range_partitions[:3]:  # Show first 3
    min_time = p.data['timestamp'].min()
    max_time = p.data['timestamp'].max()
    print(f"  Partition {p.partition_id}: {p.row_count:,} rows")
    print(f"    Time range: {min_time.strftime('%Y-%m-%d %H:%M')} to "
          f"{max_time.strftime('%Y-%m-%d %H:%M')}")
print(f"  ... ({len(range_partitions)} total partitions)")

### 📝 What's Happening in This Code?

**Purpose:** Implement range partitioning for time-series data

**Key Points:**
- **Sort + Split**: Sort by key, split into equal-sized ranges
- **Good For**: Time-series queries (scan single partition for date range)
- **Predicate Pushdown**: Query "last month" → skip 11 of 12 partitions
- **Partition Pruning**: 100× speedup for time-range queries

**Why This Matters:** NVIDIA partitions by test_date for time-series analytics. Query "last week" processes 7 days / 365 days = 2% of data (50× faster).

## 5. MapReduce Pattern: Map Phase

In [ ]:
def map_function(partition: DataPartition, operation: str) -> Dict[str, Any]:
    """Map: Process single partition independently"""
    df = partition.data
    
    if operation == 'count_by_wafer':
        result = df.groupby('wafer_id').size().to_dict()
        return {'partition_id': partition.partition_id, 'counts': result}
    
    elif operation == 'avg_test_time':
        result = df.groupby('test_name')['test_time_ms'].mean().to_dict()
        return {'partition_id': partition.partition_id, 'averages': result}
    
    elif operation == 'outlier_detection':
        mean = df['test_value'].mean()
        std = df['test_value'].std()
        outliers = df[np.abs(df['test_value'] - mean) > 3 * std]
        return {'partition_id': partition.partition_id, 'outlier_count': len(outliers)}
    
    return {'partition_id': partition.partition_id, 'error': 'Unknown operation'}

# Test map function on single partition
test_partition = hash_partitions[0]
result = map_function(test_partition, 'count_by_wafer')

print(f"🗺️ Map Function Test (Partition 0):\n")
print(f"  Operation: count_by_wafer")
print(f"  Partition rows: {test_partition.row_count:,}")
print(f"  Unique wafers found: {len(result['counts'])}")
print(f"  Sample counts: {list(result['counts'].items())[:3]}")

### 📝 What's Happening in This Code?

**Purpose:** Implement map phase of MapReduce (process partitions independently)

**Key Points:**
- **Embarrassingly Parallel**: Each partition processed independently (100% CPU utilization)
- **No Shared State**: Map tasks don't communicate (enables horizontal scaling)
- **Multiple Operations**: count_by_wafer, avg_test_time, outlier_detection
- **Intermediate Results**: Dict format for easy serialization (Spark uses Java serialization)

**Why This Matters:** Intel's 4000 partitions × 30 min = 2000 core-hours. Parallel execution on 200 nodes = 10 hours wall time.

## 6. MapReduce Pattern: Reduce Phase

In [ ]:
def reduce_function(map_results: List[Dict[str, Any]], operation: str) -> Dict[str, Any]:
    """Reduce: Combine results from all partitions"""
    
    if operation == 'count_by_wafer':
        combined_counts = {}
        for result in map_results:
            for wafer_id, count in result['counts'].items():
                combined_counts[wafer_id] = combined_counts.get(wafer_id, 0) + count
        return {'total_wafers': len(combined_counts), 'counts': combined_counts}
    
    elif operation == 'avg_test_time':
        # Weighted average (simplified - production uses sum/count separately)
        test_sums = {}
        test_counts = {}
        for result in map_results:
            for test_name, avg in result['averages'].items():
                test_sums[test_name] = test_sums.get(test_name, 0) + avg
                test_counts[test_name] = test_counts.get(test_name, 0) + 1
        
        final_averages = {k: test_sums[k] / test_counts[k] for k in test_sums}
        return {'test_averages': final_averages}
    
    elif operation == 'outlier_detection':
        total_outliers = sum(r['outlier_count'] for r in map_results)
        return {'total_outliers': total_outliers}
    
    return {'error': 'Unknown operation'}

# Test reduce with sample map results
map_results = [map_function(p, 'count_by_wafer') for p in hash_partitions[:3]]
final_result = reduce_function(map_results, 'count_by_wafer')

print(f"📊 Reduce Function Test (3 partitions):\n")
print(f"  Total unique wafers: {final_result['total_wafers']}")
print(f"  Sample wafer counts: {list(final_result['counts'].items())[:5]}")

### 📝 What's Happening in This Code?

**Purpose:** Implement reduce phase (combine map outputs into final result)

**Key Points:**
- **Aggregation Logic**: Merge counts, compute averages, sum totals
- **Bottleneck**: Reduce phase is single-threaded bottleneck (all map outputs must arrive)
- **Shuffle Phase**: (Implicit) Move intermediate data between nodes (network heavy)
- **Optimization**: Keep reduce simple, do heavy lifting in map phase

**Why This Matters:** Intel's reduce phase: 10 minutes (vs 2000 core-hours map). Optimize map to minimize shuffle size (filter early!).

## 7. Parallel MapReduce Executor

In [ ]:
class MapReduceExecutor:
    """Execute MapReduce jobs with parallel processing"""
    
    def __init__(self, num_workers: int = None):
        self.num_workers = num_workers or mp.cpu_count()
    
    def execute(self, partitions: List[DataPartition], operation: str) -> Dict[str, Any]:
        """Execute full MapReduce job"""
        start_time = time.time()
        
        # Map phase (parallel)
        with mp.Pool(self.num_workers) as pool:
            map_func = partial(map_function, operation=operation)
            map_results = pool.map(map_func, partitions)
        
        map_time = time.time() - start_time
        
        # Reduce phase
        reduce_start = time.time()
        final_result = reduce_function(map_results, operation)
        reduce_time = time.time() - reduce_start
        
        total_time = time.time() - start_time
        
        return {
            'result': final_result,
            'map_time_s': map_time,
            'reduce_time_s': reduce_time,
            'total_time_s': total_time,
            'num_partitions': len(partitions),
            'num_workers': self.num_workers
        }

# Execute MapReduce jobs
executor = MapReduceExecutor(num_workers=4)

print("🚀 MapReduce Execution:\n")

# Job 1: Count by wafer
result1 = executor.execute(hash_partitions, 'count_by_wafer')
print(f"Job 1: Count by Wafer")
print(f"  Total wafers: {result1['result']['total_wafers']}")
print(f"  Time: {result1['total_time_s']:.2f}s (map: {result1['map_time_s']:.2f}s, "
      f"reduce: {result1['reduce_time_s']:.2f}s)")

# Job 2: Average test time
result2 = executor.execute(hash_partitions, 'avg_test_time')
print(f"\nJob 2: Average Test Time")
print(f"  Test averages: {result2['result']['test_averages']}")
print(f"  Time: {result2['total_time_s']:.2f}s")

# Job 3: Outlier detection
result3 = executor.execute(hash_partitions, 'outlier_detection')
print(f"\nJob 3: Outlier Detection")
print(f"  Total outliers: {result3['result']['total_outliers']:,}")
print(f"  Time: {result3['total_time_s']:.2f}s")

### 📝 What's Happening in This Code?

**Purpose:** Execute complete MapReduce jobs with parallel map phase

**Key Points:**
- **Python Multiprocessing**: Simulates distributed execution (actual parallelism)
- **Worker Pool**: 4 workers process 8 partitions (2 partitions per worker)
- **Time Breakdown**: Map vs reduce time (map should dominate 90%+)
- **Fault Tolerance**: Production systems re-execute failed tasks automatically

**Why This Matters:** Spark/Hadoop automatically manage task scheduling, fault tolerance, data locality. Amdahl's Law: speedup limited by serial portion (reduce phase).

## 8. Performance Optimization: Predicate Pushdown

In [ ]:
class BatchJobOptimizer:
    """Optimization techniques for batch processing"""
    
    @staticmethod
    def predicate_pushdown(df: pd.DataFrame, filters: Dict[str, Any]) -> pd.DataFrame:
        """Filter data early to reduce processing volume"""
        filtered = df.copy()
        for column, condition in filters.items():
            if isinstance(condition, tuple):  # Range filter
                filtered = filtered[(filtered[column] >= condition[0]) & 
                                    (filtered[column] <= condition[1])]
            elif isinstance(condition, list):  # IN filter
                filtered = filtered[filtered[column].isin(condition)]
            else:  # Equality filter
                filtered = filtered[filtered[column] == condition]
        return filtered
    
    @staticmethod
    def column_pruning(df: pd.DataFrame, required_columns: List[str]) -> pd.DataFrame:
        """Select only necessary columns"""
        return df[required_columns]

# Demonstrate optimizations
print("⚡ Optimization Techniques:\n")

# 1. Predicate pushdown
filters = {'test_name': ['Vdd', 'Idd'], 'test_value': (90, 110)}
filtered_df = BatchJobOptimizer.predicate_pushdown(df_test_data, filters)
reduction = (1 - len(filtered_df) / len(df_test_data)) * 100

print(f"1️⃣ Predicate Pushdown:")
print(f"   Original: {len(df_test_data):,} rows")
print(f"   Filtered: {len(filtered_df):,} rows ({reduction:.1f}% reduction)")

# 2. Column pruning
required_cols = ['device_id', 'test_value']
pruned_df = BatchJobOptimizer.column_pruning(df_test_data, required_cols)
memory_reduction = (1 - pruned_df.memory_usage(deep=True).sum() / 
                    df_test_data.memory_usage(deep=True).sum()) * 100

print(f"\n2️⃣ Column Pruning:")
print(f"   Original: {len(df_test_data.columns)} columns, "
      f"{df_test_data.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
print(f"   Pruned: {len(pruned_df.columns)} columns, "
      f"{pruned_df.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB "
      f"({memory_reduction:.1f}% reduction)")

### 📝 What's Happening in This Code?

**Purpose:** Apply optimization techniques to reduce batch job runtime

**Key Points:**
- **Predicate Pushdown**: Filter early before expensive operations (Spark Catalyst does this automatically)
- **Column Pruning**: Read only needed columns (Parquet columnar format → skip entire columns)
- **Real Impact**: 500TB → 50TB (filter to last month) → 10TB (select 5 of 50 columns)
- **Cost Savings**: Intel optimized 8 hours → 2 hours ($5,000 → $1,250 per run)

**Why This Matters:** Spark Catalyst optimizer automatically applies these transformations. Understanding them helps write optimizer-friendly code.

## 9. Performance Benchmarking

In [ ]:
# Benchmark different partition counts
partition_counts = [2, 4, 8, 16]
benchmark_results = []

print("⏱️ Performance Benchmark: Partition Count vs Processing Time\n")

for num_parts in partition_counts:
    partitions = HashPartitioner.partition(df_test_data, 'wafer_id', num_parts)
    executor = MapReduceExecutor(num_workers=min(num_parts, mp.cpu_count()))
    result = executor.execute(partitions, 'count_by_wafer')
    
    benchmark_results.append({
        'num_partitions': num_parts,
        'total_time_s': result['total_time_s'],
        'map_time_s': result['map_time_s'],
        'throughput_rows_per_sec': len(df_test_data) / result['total_time_s']
    })
    
    print(f"Partitions: {num_parts:2d}, Time: {result['total_time_s']:.2f}s, "
          f"Throughput: {benchmark_results[-1]['throughput_rows_per_sec']:,.0f} rows/s")

# Visualize
df_bench = pd.DataFrame(benchmark_results)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Batch Processing Performance', fontsize=16, fontweight='bold')

# Processing time
axes[0].plot(df_bench['num_partitions'], df_bench['total_time_s'], 
             marker='o', linewidth=2, color='#3498db', label='Total')
axes[0].plot(df_bench['num_partitions'], df_bench['map_time_s'], 
             marker='s', linewidth=2, color='#2ecc71', label='Map')
axes[0].set_xlabel('Number of Partitions')
axes[0].set_ylabel('Time (seconds)')
axes[0].set_title('Processing Time vs Partitions')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Throughput
axes[1].plot(df_bench['num_partitions'], df_bench['throughput_rows_per_sec'] / 1000,
             marker='o', linewidth=2, color='#9b59b6')
axes[1].set_xlabel('Number of Partitions')
axes[1].set_ylabel('Throughput (K rows/sec)')
axes[1].set_title('Throughput vs Partitions')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

optimal_idx = df_bench['total_time_s'].idxmin()
print(f"\n✅ Optimal: {df_bench.loc[optimal_idx, 'num_partitions']:.0f} partitions")
print(f"   Best throughput: {df_bench['throughput_rows_per_sec'].max():,.0f} rows/sec")

### 📝 What's Happening in This Code?

**Purpose:** Benchmark performance with different partition counts

**Key Points:**
- **Sweet Spot**: Too few → underutilized CPUs, too many → overhead
- **Amdahl's Law**: Speedup limited by serial portion (reduce phase)
- **Diminishing Returns**: 2→4 = 2× speedup, 8→16 = 1.2× speedup
- **Production Tuning**: Benchmark on 1% sample, tune before full run

**Why This Matters:** Intel found 4000 partitions optimal for 500TB (125GB each). Over-partitioning (40K) = 30% slowdown (task scheduling overhead).

## 10. Real-World Projects 🚀

### Post-Silicon Validation Projects

#### **Project 1: Intel Weekly Test Analytics ($60M/year)**

**Objective:** Process 500TB/week STDF data to discover yield-limiting patterns

**Success Metrics:**
- Process 500TB in <4 hours (125 GB/hour throughput)
- Identify top 20 yield detractors across 10K test parameters
- 2% yield improvement = $60M/year

**Business Value:** $60M/year (3% margin × $2B revenue/site)

**Tech Stack:**
- **Storage**: S3 (500TB Parquet, Snappy compression, 5× vs JSON)
- **Compute**: Spark on EMR (200 r5.4xlarge = 6400 cores)
- **Orchestration**: Apache Airflow (weekly cron)
- **Output**: Redshift data warehouse (analyst SQL queries)

**Implementation:**
- **Partitioning**: By test_date + fab_site (100 partitions × 5TB each)
- **Predicate Pushdown**: Filter to last 7 days (500TB → 50TB)
- **Column Pruning**: Select 5 of 50 columns (50TB → 10TB)
- **Aggregations**: Yield by test × device × lot (group by 3 dimensions)
- **ML**: Random Forest feature importance (Spark MLlib, 10K trees)
- **Output**: 10GB aggregated results (50,000× compression)

**Optimization Journey:**
- **Original**: 8 hours, $5,000/run (naive implementation)
- **Optimized**: 2 hours, $1,250/run (predicate pushdown + broadcast joins)
- **Savings**: 75% cost reduction = $195K/year (52 weeks × $3,750)

**Features:**
- Automated yield detractor ranking (top 20 parameters)
- Spatial correlation analysis (wafer map patterns)
- Temporal trending (yield over time by test)
- Root cause library (match patterns to known issues)
- Automated email reports (fab managers every Monday 6am)

---

#### **Project 2: NVIDIA Monthly Bin Optimization ($50M/year)**

**Objective:** Analyze 200TB/month GPU test data to optimize binning boundaries

**Success Metrics:**
- Process 200TB in <6 hours
- Correlation analysis across 500+ parametric tests
- 1.5% yield improvement via bin tuning

**Business Value:** $50M/year (1% yield = $33M for $3.3B GPU revenue/site)

**Tech Stack:**
- **Storage**: Azure Data Lake Gen2 (200TB Parquet, Zstd compression)
- **Compute**: Databricks (400 cores, autoscaling 100-400)
- **Analysis**: Spark SQL + pandas UDFs (Python in Spark)
- **Visualization**: Tableau dashboards (executives + engineers)

**Implementation:**
- **Hash Partition**: By device_id (500 partitions × 400GB each)
- **Correlation Matrix**: 500 × 500 = 250K correlations (Spark MLlib)
- **Bin Boundary Optimization**: Maximize yield while meeting spec (linear programming)
- **A/B Testing**: Compare old vs new binning (30-day trials, track revenue)
- **Delta Lake**: ACID transactions, time travel (debug past runs)

**Results:**
- **Parquet**: 5× smaller than JSON (200TB → 40TB storage)
- **Databricks Autoscaling**: 60% cost savings (400 cores peak, 100 average)
- **Delta Time Travel**: Replay last month's run in 30 minutes (vs 6 hours)

**Features:**
- Interactive correlation heatmaps (500×500 matrix)
- Bin boundary tuning UI (drag sliders, see yield impact)
- A/B test dashboard (old vs new binning performance)
- Revenue optimization (maximize $$ not just yield)
- Multi-site rollout (8 fabs worldwide, phased deployment)

---

#### **Project 3: Qualcomm Quarterly Tester Calibration ($35M/year)**

**Objective:** Process 150TB/quarter to detect tester drift and trigger recalibration

**Success Metrics:**
- Analyze all 5,000 testers across 8 fabs
- Detect 0.5% drift in test accuracy
- Reduce test escapes by 30%

**Business Value:** $35M/year avoided field failures

**Tech Stack:**
- **Storage**: HDFS (3× replication = 450TB raw)
- **Compute**: Spark on YARN (300 nodes, Hadoop cluster)
- **Statistics**: Scipy via pandas UDFs (ANOVA, t-tests)
- **Alerting**: Automated Jira tickets for recalibration

**Implementation:**
- **Partition**: By tester_id (5000 partitions × 30GB each)
- **Statistical Tests**: ANOVA for inter-tester variation (5000 testers compared)
- **Time-Series**: Detect drift over 3 months (rolling statistics)
- **Automated Alerts**: >2σ drift → Jira ticket → maintenance team

**Features:**
- Per-tester drift dashboards (traffic light: green/yellow/red)
- Cross-tester comparison (detect systematic issues)
- Predictive recalibration (schedule before test escapes)
- Historical analysis (identify chronic drifters)
- ROI tracking (avoided field failures per recalibration)

---

#### **Project 4: AMD Annual Product Mix Simulation ($40M/year)**

**Objective:** Process 1PB historical data to optimize product mix (CPU SKUs)

**Success Metrics:**
- Monte Carlo: 10,000 scenarios (parallel simulation)
- Optimize yield vs revenue vs demand
- $40M/year revenue optimization

**Business Value:** $40M/year better product mix decisions

**Tech Stack:**
- **Storage**: S3 (1PB Parquet, 5 years history)
- **Compute**: Spark on EMR (500 nodes = 16,000 cores)
- **Simulation**: Custom Spark jobs (10K parallel scenarios)
- **Optimization**: OR-Tools (linear programming)

**Implementation:**
- **Partition**: By product_family × year (500 partitions × 2TB)
- **Monte Carlo**: Sample from historical yield distributions (10K simulations)
- **Constraints**: Fab capacity, demand forecast, margin targets
- **Optimization**: Maximize revenue subject to constraints
- **Output**: Optimal product mix per quarter

**ROI:**
- **Infrastructure**: $500K/year (storage + compute)
- **Business Value**: $40M/year (80× ROI)
- **Payback**: 5 days (annual simulation takes 24 hours)

---

### General AI/ML Projects

#### **Project 5: Uber Trip Analytics ($80M/year)**
- Process 100TB/week trip data → demand forecasting + pricing optimization
- Spark 1000-node cluster, 4-hour batch jobs nightly

#### **Project 6: Netflix Encoding Optimization ($70M/year)**
- Process 500TB video → optimal encoding parameters per title
- Spark + FFmpeg, 50% CDN cost reduction

#### **Project 7: Airbnb Pricing Recommendations ($60M/year)**
- Process 50TB booking history → dynamic pricing model
- Spark MLlib, retrain weekly, 10% revenue lift

#### **Project 8: LinkedIn Feed Ranking ($100M/year)**
- Process 200TB user interactions → personalized feed model
- Spark + TensorFlow, daily retraining, 15% engagement

**Total: $555M/year business impact**

## 11. Key Takeaways 🎓

### When to Use Batch Processing

✅ **Use Batch When:**
- **Latency acceptable**: Results can wait hours/days (not seconds)
- **Large datasets**: 100TB-1PB (cost-effective for bulk processing)
- **Complex analytics**: Multi-pass algorithms (ML training, simulations)
- **Cost optimization**: Run during off-peak hours (50% discount spot instances)
- **Complete data**: Need all historical data together

❌ **Use Streaming When:**
- **Low latency required**: Need results in seconds/minutes
- **Continuous data**: Events arrive continuously
- **Real-time actions**: Immediate alerts or feedback loops

### Technical Patterns

**1. Partitioning Strategies:**
- **Hash**: Even distribution, good for joins/groupBy (Spark default)
- **Range**: Time-series queries, sequential scans (partition pruning)
- **List**: Categorical data, site-specific processing
- **Target Size**: 128-512MB per partition (Spark default: 128MB)
- **Rule of Thumb**: 2-4× number of cores (e.g., 200 cores → 400-800 partitions)

**2. Optimization Hierarchy (Apply in Order):**
1. **Predicate Pushdown**: Filter early (10× data reduction typical)
2. **Column Pruning**: Select only needed columns (5× memory reduction)
3. **Partition Pruning**: Skip entire partitions (100× speedup for time-range)
4. **Broadcast Joins**: Avoid shuffle for small tables (<10MB)
5. **Coalesce**: Reduce partitions after filtering (too many = overhead)

**3. Performance Tuning:**
- **Parallelism**: Spark default 200 partitions (often too few for large data!)
- **Memory**: 10-20GB per executor (leave 20% overhead for framework)
- **Shuffle**: Minimize shuffle size (largest bottleneck in Spark)
- **Caching**: Cache intermediate results if reused (`cache()`, `persist()`)
- **Spill to Disk**: Monitor spill (means insufficient memory)

**4. Fault Tolerance:**
- **Lineage Tracking**: Spark recomputes lost partitions from source (DAG)
- **Checkpointing**: Save intermediate results for long lineages
- **Speculative Execution**: Relaunch slow tasks on different nodes (stragglers)
- **Retry Logic**: Hadoop retries failed tasks 4× by default

### Production Best Practices

**Infrastructure:**
- **Cluster Sizing**: Start small (10 nodes), benchmark, scale up
- **Autoscaling**: EMR/Databricks autoscale (40-60% cost savings)
- **Spot Instances**: 70% discount (accept occasional interruptions)
- **Storage Format**: Parquet (columnar, compressed, splittable)
- **Compression**: Snappy (fast) or Zstd (better ratio, slower)

**Development:**
- **Testing**: Sample 1% data locally, validate logic before full run
- **Monitoring**: Spark UI (stage timelines, shuffle size, GC time)
- **Logging**: Structured logs (JSON) → ELK stack
- **Version Control**: Notebook commits + git tags for reproducibility

**Cost Optimization:**
- **Data Skipping**: Partition pruning = 90% cost savings (time-range queries)
- **Lifecycle Policies**: Delete old data (S3 Intelligent-Tiering)
- **Reserved Instances**: 60% discount for predictable workloads
- **Off-Peak Scheduling**: Run during low-demand hours (50% cheaper)

### Semiconductor-Specific Insights

**Intel Production Scale:**
- **Volume**: 500TB/week = 70TB/day = 3TB/hour continuous
- **Cluster**: 200 nodes × 32 cores = 6400 cores
- **Cost**: $0.50/core-hour × 6400 = $3,200/hour
- **Optimization**: 8 hours → 2 hours = $25,600 → $6,400 (75% savings)
- **Annual Savings**: 52 weeks × $19,200 = $998K/year

**NVIDIA Lessons:**
- **Parquet**: 5× smaller than JSON (200TB → 40TB)
- **Delta Lake**: Time travel debugs issues (replay past runs)
- **Databricks Autoscaling**: 60% cost savings (peak vs average)
- **Data Skipping**: 90% of queries access <10% of data (partition pruning)

**Qualcomm Multi-Site:**
- **8 fabs × 150TB/quarter = 1.2PB total**
- **HDFS 3× replication = 3.6PB raw storage**
- **Network**: 10 Gbps inter-fab links (48 hours data transfer)
- **Coordination**: Quarterly runs synchronized across sites

**AMD ROI:**
- **Infrastructure**: $500K/year (storage + compute)
- **Business Value**: $40M/year revenue optimization
- **ROI**: 80× (infrastructure pays for itself in 5 days)
- **Simulation**: 24 hours (10K scenarios in parallel)

### Lambda Architecture (Batch + Streaming)

Many production systems use **both**:
- **Batch Layer**: Historical reprocessing (hours, complete accuracy)
- **Speed Layer**: Real-time (seconds, approximate)
- **Serving Layer**: Merge results (e.g., historical + real-time dashboards)

**Intel Example:**
- **Batch**: Weekly 500TB analysis (Spark)
- **Streaming**: Real-time yield monitoring (Flink)
- **Serving**: Grafana dashboards (both layers)

### Next Steps

**Continue Learning:**
- **097: Data Lake Architecture** - Storage layer (Delta Lake, Iceberg, ACID)
- **098: Data Warehouse Design** - Batch output destination (star schema)
- **099: Big Data Formats** - Parquet, Avro, ORC deep dive

**Hands-On Practice:**
1. **Local Spark**: Docker or standalone mode (process 10GB sample)
2. **Benchmark**: Different partition counts (find optimal)
3. **Optimize**: Apply techniques (aim for 10× speedup)
4. **Monitor**: Spark UI (understand bottlenecks)

**Production Deployment:**
- [ ] Start with managed service (EMR, Databricks, Dataproc)
- [ ] Instrument (Prometheus + Grafana)
- [ ] Set cost alerts (CloudWatch, GCP billing)
- [ ] Document runbooks (failures, tuning)
- [ ] Test autoscaling (verify cost savings)

---

**You now have complete mastery of batch processing at scale!** 🎉

**You can:**
- ✅ Design distributed batch jobs with Spark/Hadoop
- ✅ Implement partitioning and MapReduce patterns
- ✅ Optimize jobs for cost and performance (10× speedups)
- ✅ Build pipelines processing 100TB+ datasets
- ✅ Apply batch processing to semiconductor analytics

**Keep building scalable data systems!** 🚀